In [25]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.optimizers import SGD
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline

### Ce fichier se sert des réseaux de neurones pour faire son apprentissage

In [26]:
df = pd.read_csv("kobe.csv")
data = df.dropna() # Suppression des lignes(les tirs) dont nous ne connaissons pas l'issue(marqué ou manqué)
categorical_vars = ["action_type", "combined_shot_type", "season", "shot_type", "shot_zone_area",
                     "shot_zone_basic", "shot_zone_range", "team_name", "game_date", "matchup", "opponent"]
numerical_vars = [col for col in df.columns if col not in categorical_vars + ["shot_made_flag", "shot_id", 'team_id']]  # garder le shot_id ?
X = data[numerical_vars].values.astype(float)
y = data["shot_made_flag"]  # Target : issue du tir 

In [27]:
dummy_y = np_utils.to_categorical(y)

In [28]:
# define baseline model
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(400, input_dim=11, activation='relu'))
	model.add(Dense(400, input_dim=11, activation='relu'))
	model.add(Dense(2, activation='softmax'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [29]:
def processing_data(X):

    # create scaler
    scaler = StandardScaler()
    # fit and transform in one step
    standardized = scaler.fit_transform(X)
    # inverse transform
    inverse = scaler.inverse_transform(standardized)
    
    return standardized

In [30]:
X = processing_data(X)
X_train, X_test, y_train, y_test = train_test_split(X,dummy_y,test_size=0.2, random_state=0)

In [31]:
print(y_train.shape, y_test.shape, X_train.shape, X_test.shape)

(20557, 2) (5140, 2) (20557, 11) (5140, 11)


In [24]:
y_test.reshape(y_test.shape[0], 1)
y_train.reshape(y_train.shape[0], 1)

AttributeError: 'Series' object has no attribute 'reshape'

In [32]:
opt = SGD(learning_rate=0.1, momentum=0.9) # descente de gradient 
model = create_model()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=150)

Epoch 1/150
643/643 [==============================] - 6s 4ms/step - loss: 0.6763 - accuracy: 0.5877
Epoch 2/150
643/643 [==============================] - 2s 3ms/step - loss: 0.6761 - accuracy: 0.5883
Epoch 3/150
643/643 [==============================] - 2s 3ms/step - loss: 0.6711 - accuracy: 0.5996
Epoch 4/150
643/643 [==============================] - 2s 3ms/step - loss: 0.6721 - accuracy: 0.5964
Epoch 5/150
643/643 [==============================] - 2s 3ms/step - loss: 0.6718 - accuracy: 0.5986
Epoch 6/150
643/643 [==============================] - 2s 3ms/step - loss: 0.6694 - accuracy: 0.5988
Epoch 7/150
643/643 [==============================] - 2s 3ms/step - loss: 0.6707 - accuracy: 0.5991
Epoch 8/150
643/643 [==============================] - 2s 3ms/step - loss: 0.6699 - accuracy: 0.5980
Epoch 9/150
643/643 [==============================] - 2s 3ms/step - loss: 0.6680 - accuracy: 0.6022
Epoch 10/150
643/643 [==============================] - 2s 3ms/step - loss: 0.6696 - accura

In [33]:
results, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Accuracy du modèle : {acc*100}")

Accuracy du modèle : 57.49027132987976


In [36]:
prediction= model.predict(X_test)

161/161 [==============================] - 0s 1ms/step


In [6]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=3)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

C:\Users\guilh\AppData\Local\Temp\ipykernel_26308\620187425.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=2)


Epoch 1/200
4626/4626 - 15s - loss: 33383.2695 - accuracy: 0.5498 - 15s/epoch - 3ms/step
Epoch 2/200
4626/4626 - 12s - loss: 0.6873 - accuracy: 0.5545 - 12s/epoch - 3ms/step
Epoch 3/200
